In [2]:
import xarray as xr
import getpass
from pydap.client import open_url
from pydap.cas.get_cookies import setup_session

## OPeNDAP for Copernicus data

In [12]:
USERNAME = 'mphan'
PASSWORD = getpass.getpass('Enter your password: ')

Enter your password:  ········


In [14]:
# change your Dataset ID accordingly
DATASET_ID = 'global-analysis-forecast-bio-001-028-daily'

In [15]:
def copernicusmarine_datastore(dataset, username, password):
    cas_url = 'https://cmems-cas.cls.fr/cas/login'
    session = setup_session(cas_url, username, password)
    try:
        session.cookies.set("CASTGC", session.cookies.get_dict()['CASTGC'])
    except:
        print("Bad credentials. Please try again.")
    database = ['my', 'nrt']
    url = f'https://{database[0]}.cmems-du.eu/thredds/dodsC/{dataset}'

    try:
        data_store = xr.backends.PydapDataStore(open_url(url, session=session, user_charset='utf-8')) # needs PyDAP >= v3.3.0 see https://github.com/pydap/pydap/pull/223/commits 
    except:
        url = f'https://{database[1]}.cmems-du.eu/thredds/dodsC/{dataset}'
        data_store = xr.backends.PydapDataStore(open_url(url, session=session, user_charset='utf-8')) # needs PyDAP >= v3.3.0 see https://github.com/pydap/pydap/pull/223/commits
    
    return data_store

In [16]:
data_store = copernicusmarine_datastore(DATASET_ID, USERNAME, PASSWORD)

In [17]:
DS = xr.open_dataset(data_store)

In [18]:
DS

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 681, depth: 50, time: 1021)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * latitude   (latitude) float32 -80.0 -79.75 -79.5 -79.25 ... 89.5 89.75 90.0
  * depth      (depth) float32 0.494 1.541 2.646 ... 5.275e+03 5.728e+03
  * time       (time) datetime64[ns] 2020-11-01T12:00:00 ... 2023-08-18T12:00:00
Data variables:
    talk       (time, depth, latitude, longitude) float32 ...
    chl        (time, depth, latitude, longitude) float32 ...
    dissic     (time, depth, latitude, longitude) float32 ...
    fe         (time, depth, latitude, longitude) float32 ...
    no3        (time, depth, latitude, longitude) float32 ...
    nppv       (time, depth, latitude, longitude) float32 ...
    o2         (time, depth, latitude, longitude) float32 ...
    ph         (time, depth, latitude, longitude) float32 ...
    phyc       (time, depth, latitude, longitude) float32 ...
    po4        (time, depth, latitude, longitude) float32 ...
    si         (time, depth, latitude, longitude) float32 ...
    spco2      (time, latitude, longitude) float32 ...
Attributes: (12/16)
    product:                       GLOBAL_ANALYSIS_FORECAST_BIO_001_028
    producer:                      CMEMS - Global Monitoring and Forecasting ...
    title:                         daily mean fields from Global Ocean Biogeo...
    area:                          GLOBAL
    quality_information_document:  http://marine.copernicus.eu/documents/QUID...
    Conventions:                   CF-1.6
    ...                            ...
    licence:                       http://marine.copernicus.eu/services-portf...
    dataset:                       global-analysis-forecast-bio-001-028-daily
    institution:                   Mercator Ocean
    product_user_manual:           http://marine.copernicus.eu/documents/PUM/...
    forecast_type:                 forecast
    bulletin_date:                 2023-08-09 00:00:00

In [19]:
DS_date = DS.sel(time=slice('2021-01-01', '2021-01-10'))

In [26]:
# we're interested in two variables: CHL (chlorophyll-a) & pH
bio_data = DS_date.sel(longitude=-122.186, latitude=36.713, depth=891, method='nearest')[['chl', 'ph']]

In [32]:
bio_data.to_dataframe()

,chl,ph,longitude,latitude,depth
time,,,,,
2021-01-01 12:00:00,0.002936,7.559717,-122.25,36.75,902.339294
2021-01-02 12:00:00,0.002936,7.559319,-122.25,36.75,902.339294
2021-01-03 12:00:00,0.002936,7.558249,-122.25,36.75,902.339294
2021-01-04 12:00:00,0.002936,7.557823,-122.25,36.75,902.339294
2021-01-05 12:00:00,0.002936,7.558003,-122.25,36.75,902.339294
2021-01-06 12:00:00,0.002936,7.558708,-122.25,36.75,902.339294
2021-01-07 12:00:00,0.002935,7.559104,-122.25,36.75,902.339294
2021-01-08 12:00:00,0.002935,7.558798,-122.25,36.75,902.339294
2021-01-09 12:00:00,0.002935,7.558220,-122.25,36.75,902.339294


As you can see, there is not much variability for the two variables between the days to guarantee a proper correlation with our drastic differences in noise recording data.